## V3
- Oneshot training without generator
- Normalise tempo

In [1]:
!pip install pretty_midi
!pip install keras_self_attention
!pip install music21
!pip install mido

You should consider upgrading via the '/home/victorpham1997/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/victorpham1997/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/victorpham1997/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/victorpham1997/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
import pretty_midi
from music21 import *
import pickle
import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import time
import os
import glob
from itertools import groupby
import mido
from mido import MidiFile, merge_tracks, tempo2bpm
import math

from keras.preprocessing import sequence
from keras.models import Sequential 
from keras.layers import Dense, LSTM, Bidirectional, Dropout, GlobalMaxPooling1D, Activation, GlobalMaxPooling2D
from keras_self_attention import SeqSelfAttention
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.layers import Layer
from tensorflow.python.client import device_lib
import keras

# data_path = "./selected_data/"
data_path = "./doug_mckenzie_midi/"
# data_path = "./sample_data/"
# encoded_data_path = "./encoded_doug_mckenzie_midi_32/"
encoded_data_path = "./encoded_doug_mckenzie_midi_16_v2/"



Using TensorFlow backend.


## Parsing Midi file as vector

In [ ]:
def extract_midi_info(path):
    mid = converter.parse(path)
    max_idx = np.argmax(len(i) for i in mid)
    piano_part = mid[max_idx] #probably
    for i in piano_part:
        if isinstance(i, tempo.MetronomeMark):
            bpm = i.getQuarterBPM()
            break
    try:
        key = piano_part.keySignature
    except:
        print(f"Error while finding key signature for song {temp}")

    key_in_major = key.asKey(mode='major')
    offset_by = key_in_major.tonic.pitchClass
    return offset_by, bpm

def preprocess_midi(path, offset_by, bpm):
    mid = pretty_midi.PrettyMIDI(midi_file=path)
    filtered_inst_ls = [inst for inst in mid.instruments if ((len(inst.notes) > 0) and
                                                    (inst.is_drum == False) and
                                                    (inst.program < 8)
                                                   )]
    piano = filtered_inst_ls[np.argmax([len(inst.notes) for inst in filtered_inst_ls])]
            
    start_time = piano.notes[0].start
    end_time = piano.get_end_time()
    
    quater_note_len = 60/bpm
#     Set 4 for 16th note, 8 for 32 note
    nth_note = 8
#     Set fs to get 16th notes
    fs = 1/(quater_note_len/nth_note)
#     fs = 100
    
    piano_roll = piano.get_piano_roll(fs = fs, times = np.arange(start_time, end_time,1./fs))
    piano_roll = np.roll(piano_roll, -offset_by)
    out = np.where(piano_roll > 0, 1,0)
    
    return out.T

def process_piano_roll(piano_roll, max_consecutive = 64):
#     This function is to remove consecutive notes that last for more than roughtly 2 secs
    prev = np.random.rand(128)
    count = 0
    remove_idxs = []
    remove_slice = []
    for idx, piano_slice in enumerate(piano_roll):
#         print(prev.shape)
#         print(piano_slice.shape)
        if(np.array_equal(prev, piano_slice)):
            count+=1
            if (count > max_consecutive):
                remove_idxs.append(idx)
                if (str(piano_slice) not in remove_slice):
                    remove_slice.append(str(piano_slice))
        else:
            count = 0
        prev = piano_slice
    out_piano_roll = np.delete(piano_roll, remove_idxs, axis=0)
    return out_piano_roll

failed_list = []
# keep track of list of midi we failed to parse and preprocess
for temp in glob.glob(data_path + "*.mid"):
    try:
        print(temp)
        offset_by, bpm = extract_midi_info(temp)
        piano_roll = preprocess_midi(temp, offset_by, bpm)
        piano_roll = process_piano_roll(piano_roll)
        name  = temp.split("/")[-1].split(".")[0]
        out_name = encoded_data_path + f'encoded_{name}.npy'
        np.save(out_name, piano_roll)
        print(f"saved {out_name}")
        
    except:
        print(f"Faield to preprocess {temp}")
        failed_list.append(temp)
        continue

./doug_mckenzie_midi/DancingOnTheCeiling.mid
removed 0 from song
saved ./encoded_doug_mckenzie_midi_16_v2/encoded_DancingOnTheCeiling.npy
./doug_mckenzie_midi/AveMaria.mid
removed 0 from song
saved ./encoded_doug_mckenzie_midi_16_v2/encoded_AveMaria.npy
./doug_mckenzie_midi/YouAndTheNightAndTheMusicRubato.mid
removed 0 from song
saved ./encoded_doug_mckenzie_midi_16_v2/encoded_YouAndTheNightAndTheMusicRubato.npy
./doug_mckenzie_midi/EasyDoesIt.mid
removed 0 from song
saved ./encoded_doug_mckenzie_midi_16_v2/encoded_EasyDoesIt.npy
./doug_mckenzie_midi/MyFoolish.mid
removed 4 from song
removed ['[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]'] from song
saved ./encoded_doug_mckenzie_midi_16_v2/encoded_MyFoolish.npy
./doug_mckenzie_midi/AllTheThingssoloandtrio.mid
removed 0 f

removed 15 from song
removed ['[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0\n 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]'] from song
saved ./encoded_doug_mckenzie_midi_16_v2/encoded_OldFolks.npy
./doug_mckenzie_midi/EastOfTheSun.mid
removed 0 from song
saved ./encoded_doug_mckenzie_midi_16_v2/encoded_EastOfTheSun.npy
./doug_mckenzie_midi/HeatherOnTheHill(Part2).mid
removed 0 from song
saved ./encoded_doug_mckenzie_midi_16_v2/encoded_HeatherOnTheHill(Part2).npy
./doug_mckenzie_midi/Sometimeago.mid
removed 0 from song
saved ./encoded_doug_mckenzie_midi_16_v2/encoded_Sometimeago.npy
./doug_mckenzie_midi/Embraceable.mid
removed 0 from song
saved ./encoded_doug_mckenzie_midi_16_v2/encoded_Embraceable.npy
./doug_mckenzie_midi/Yesterdays1.mid
removed 99 from song
removed ['[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 